# Capture the video and size it

In [ ]:
import cv2
import cvzone

cap = cv2.VideoCapture('Vid 12.mp4')

while True:
    success,img=cap.read()
    img = img[0:1700, :]
    img = cv2.resize(img, (0, 0), None, 0.45, 0.45)
    cv2.imshow("Image", img)
    cv2.waitKey(100)

# Finding the color of the ball


In [ ]:
import cv2
import cvzone
from cvzone.ColorModule import ColorFinder

cap = cv2.VideoCapture('Vid 12.mp4')

#myColorFinder = ColorFinder(True)
#hsvVals= 'orange'

myColorFinder = ColorFinder(False)
hsvVals= {'hmin': 145, 'smin': 44, 'vmin': 110, 'hmax': 179, 'smax': 255, 'vmax': 255}

while True:
    #img = cv2.imread("Ball-1.png")
    success,img=cap.read()
    img = img[0:900, :]
    
    imgColor, mask = myColorFinder.update(img,hsvVals)
    
    imgColor = cv2.resize(imgColor, (0, 0), None, 0.45, 0.45)
    #cv2.imshow("Image", img)
    cv2.imshow("ImageColor", imgColor)
    cv2.waitKey(50)

# Finding the co ordinates

In [ ]:
import cv2
import cvzone
from cvzone.ColorModule import ColorFinder
import numpy as np
import math

cap = cv2.VideoCapture('Vid 12.mp4')

myColorFinder = ColorFinder(False)
hsvVals= {'hmin': 145, 'smin': 44, 'vmin': 110, 'hmax': 179, 'smax': 255, 'vmax': 255}

posListX, posListY = [], []
xList = [item for item in range(0,1800)]

pause = False

while True:
    if not pause:
        success,img=cap.read()
        img = img[0:1700, :] 
        
        imgColor, mask = myColorFinder.update(img,hsvVals)
        imgContours, contours = cvzone.findContours(img,mask,minArea=500)
        
        imgContours = imgContours[0:1700, :]
        
        if contours:
            posListX.append(contours[0]['center'][0])
            posListY.append(contours[0]['center'][1])
            
        if posListX:
            A,B,C = np.polyfit(posListX, posListY ,2)
            
            for i, (posX, posY) in enumerate(zip(posListX, posListY)):  
                pos = (posX,posY)
                cv2.circle(imgContours, pos,7, (0,255,0), cv2.FILLED)
                if i == 0:
                    cv2.line(imgContours, pos, pos,(0,255,0),3)
                else:
                    cv2.line(imgContours, pos, (posListX[i-1],posListY[i-1]),(0,255,0),3)
                print("Ball center coordinates:", pos)
            
            for x in xList:
                y = int(A * x ** 2 + B * x + C  )    
                cv2.circle(imgContours, (x,y),5, (255,0,255), cv2.FILLED)
            a = A
            b = B
            c = C - 330
            
            if b ** 2 - (4 * a * c) > 0:
    
                x = int((-b - math.sqrt(b ** 2 - (4 * a * c))) / (2 * a))
                print("Calculated X:", x)

            else:
                continue
        
        imgContours = imgContours[0:1700, :]
    
        imgContours = cv2.resize(imgContours, (0, 0), None, 1, 1)
        #cv2.imshow("Image", img)""
        cv2.imshow("ImageColor", imgContours)
        
    key = cv2.waitKey(100)
    if key == ord(' '):  # Spacebar to pause/unpause
        pause = not pause
    elif key == 27:  # Escape key to exit
        break

cap.release()
cv2.destroyAllWindows()


# Prediction


In [1]:
import cv2
import cvzone
from cvzone.ColorModule import ColorFinder
import numpy as np
import math

cap = cv2.VideoCapture('Vid 9.mp4')

myColorFinder = ColorFinder(False)
hsvVals= {'hmin': 145, 'smin': 44, 'vmin': 110, 'hmax': 179, 'smax': 255, 'vmax': 255}

posListX, posListY = [], []
xList = [item for item in range(0,1800)]
prediction = False
frame_counter = 0
paused = False

while True:
    success,img=cap.read()
    img = img[0:1700, :] 
    
    imgColor, mask = myColorFinder.update(img,hsvVals)
    imgContours, contours = cvzone.findContours(img,mask,minArea=500)
    
    if contours:
        posListX.append(contours[0]['center'][0])
        posListY.append(contours[0]['center'][1])
        
    if posListX:
        A,B,C = np.polyfit(posListX, posListY ,2)
        
        for i, (posX, posY) in enumerate(zip(posListX, posListY)):  
            pos = (posX,posY)
            cv2.circle(imgContours, pos,5, (0,255,0), cv2.FILLED)
            if i == 0:
                cv2.line(imgContours, pos, pos,(0,255,0),4)
            else:
                cv2.line(imgContours, pos, (posListX[i-1],posListY[i-1]),(0,255,0),4)
        
        for x in xList:
            y = int(A * x ** 2 + B * x + C  )    
            cv2.circle(imgContours, (x,y),2, (255,0,255), cv2.FILLED)
            
        if len(posListX)<10:
            a = A
            b = B
            c = C - 330
            
            if b ** 2 - (4 * a * c) > 0:
                x = int((-b - math.sqrt(b ** 2 - (4 * a * c))) / (2 * a))
                prediction = 429 < x < 505
            else:
                continue

        if prediction:
            cvzone.putTextRect(imgContours, "Basket", (50,100), scale = 3, 
                               thickness=4,colorR=(0,255,0), offset = 15)
        else:
            cvzone.putTextRect(imgContours, "No Basket", (50,100), scale = 3, 
                               thickness=4,colorR=(0,0,200), offset =15)
    
    imgContours = imgContours[0:1700, :]
    imgContours = cv2.resize(imgContours, (0, 0), None, 1, 1)
    
    if not paused:
        cv2.imshow("ImageColor", imgContours)
        
    key = cv2.waitKey(100)
    
    if key == ord('q'):
        break
    elif key == ord('p'):  
        paused = not paused
    
    frame_counter += 1
    if frame_counter == 9:  
        paused = True
        
    if paused:
        key = cv2.waitKey(0) 
        if key == ord('p'):
            paused = False
            
cap.release()
cv2.destroyAllWindows()


C:\Users\tanvi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3457: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\tanvi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3457: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


TypeError: 'NoneType' object is not subscriptable